
# 🎨 Neural Style **Transfer**


In [46]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import torchvision.models as models
import copy
import numpy as np


# Upload images

In [47]:
def load_image(image_path, max_size=256):
    image = Image.open(image_path).convert('RGB')

    if max_size:
        if max(image.size) > max_size:
            size = max_size
            image = image.resize((size, int(size * image.size[1] / image.size[0])), Image.LANCZOS)

    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])

    image = transform(image).unsqueeze(0)  #batch
    return image


2.1 Load the pre-trained VGG19 model

In [48]:
def get_features(image, model, layers=None):
    """استخراج خصائص الصورة من طبقات محددة في النموذج"""
    if layers is None:
        layers = {'0': 'conv1_1',
                  '5': 'conv2_1',
                  '10': 'conv3_1',
                  '19': 'conv4_1',
                  '21': 'conv4_2',
                  '28': 'conv5_1'}

    features = {}
    x = image
    for name, layer in model._modules.items():
        x = layer(x)
        if name in layers:
            features[layers[name]] = x

    return features


2.2 تعريف دوال حساب الخسارة (Content Loss & Style Loss)


In [49]:
def gram_matrix(tensor):
    """حساب مصفوفة غرام للتنسور"""
    batch_size, channels, h, w = tensor.size()
    tensor = tensor.view(batch_size * channels, h * w)
    gram = torch.mm(tensor, tensor.t())
    return gram.div(channels * h * w)

def content_loss(target_features, content_features):
    """حساب خسارة المحتوى"""
    return F.mse_loss(target_features, content_features)

def style_loss(target_features, style_features):
    """حساب خسارة الأسلوب"""
    target_gram = gram_matrix(target_features)
    style_gram = gram_matrix(style_features)
    return F.mse_loss(target_gram, style_gram)


In [50]:
def style_transfer(content_image, style_image, num_steps=300,
                   style_weight=1e4, content_weight=1):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    content_image = content_image.to(device)
    style_image = style_image.to(device)

    # model VGG
    vgg = models.vgg19(pretrained=True).features
    for param in vgg.parameters():
        param.requires_grad_(False)
    vgg.to(device)

    content_layers = ['conv4_2']
    style_layers = ['conv1_1', 'conv2_1', 'conv3_1', 'conv4_1', 'conv5_1']

    # gradient
    target_image = content_image.clone().requires_grad_(True).to(device)

    optimizer = optim.Adam([target_image], lr=0.01)

    content_features = get_features(content_image, vgg)
    style_features = get_features(style_image, vgg)

    for step in range(1, num_steps + 1):
        optimizer.zero_grad()

        target_features = get_features(target_image, vgg)

        c_loss = 0
        for layer in content_layers:
            c_loss += content_loss(target_features[layer], content_features[layer])

        s_loss = 0
        for layer in style_layers:
            s_loss += style_loss(target_features[layer], style_features[layer])

        total_loss = content_weight * c_loss + style_weight * s_loss
        total_loss.backward()
        optimizer.step()

        if step % 50 == 0:
            print(f"Step {step}: Content Loss = {c_loss.item():.4f}, Style Loss = {s_loss.item():.4f}")

    target_image.data.clamp_(0, 1)
    return target_image


2.4 Try a fast expermant (optional)


In [51]:
class TransformerNet(nn.Module):
    """شبكة التحويل للنقل الأسلوبي السريع"""
    def __init__(self):
        super(TransformerNet, self).__init__()

        # طبقات الترميز (Encoder)
        self.enc1 = nn.Conv2d(3, 32, kernel_size=9, stride=1, padding=4)
        self.enc2 = nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1)
        self.enc3 = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1)

        # الكتل المتبقية (Residual Blocks)
        self.res1 = ResidualBlock(128)
        self.res2 = ResidualBlock(128)
        self.res3 = ResidualBlock(128)
        self.res4 = ResidualBlock(128)
        self.res5 = ResidualBlock(128)

        # طبقات فك الترميز (Decoder)
        self.dec1 = nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.dec2 = nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.dec3 = nn.Conv2d(32, 3, kernel_size=9, stride=1, padding=4)

    def forward(self, x):
        # تمرير أمامي
        x = F.relu(self.enc1(x))
        x = F.relu(self.enc2(x))
        x = F.relu(self.enc3(x))

        x = self.res1(x)
        x = self.res2(x)
        x = self.res3(x)
        x = self.res4(x)
        x = self.res5(x)

        x = F.relu(self.dec1(x))
        x = F.relu(self.dec2(x))
        x = self.dec3(x)

        return x

class ResidualBlock(nn.Module):
    """كتلة متبقية للنقل الأسلوبي السريع"""
    def __init__(self, channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(channels, channels, kernel_size=3, stride=1, padding=1)
        self.in1 = nn.InstanceNorm2d(channels, affine=True)
        self.conv2 = nn.Conv2d(channels, channels, kernel_size=3, stride=1, padding=1)
        self.in2 = nn.InstanceNorm2d(channels, affine=True)

    def forward(self, x):
        residual = x
        x = F.relu(self.in1(self.conv1(x)))
        x = self.in2(self.conv2(x))
        return x + residual


3.1 optional


In [52]:
def experiment_with_weights(content_image, style_image):
    """تجربة قيم مختلفة لأوزان المحتوى والأسلوب"""
    content_weights = [1, 10, 100]
    style_weights = [1e4, 1e5, 1e6, 1e7]

    results = {}

    for c_weight in content_weights:
        for s_weight in style_weights:
            print(f"Testing with content_weight={c_weight}, style_weight={s_weight}")
            result = style_transfer(
                content_image,
                style_image,
                num_steps=200,
                content_weight=c_weight,
                style_weight=s_weight
            )
            results[(c_weight, s_weight)] = result

    return results


4.1 to visulaise the orignal imgaes(opintail)


In [53]:
def display_results(original, style, results):
    """عرض الصور الأصلية والمحولة جنبًا إلى جنب"""
    fig, ax = plt.subplots(3, 3, figsize=(15, 15))

    ax[0, 0].imshow(to_pil_image(original[0]))
    ax[0, 0].set_title("Original Content")
    ax[0, 1].imshow(to_pil_image(style[0]))
    ax[0, 1].set_title("Style Reference")
    ax[0, 2].axis('off')

    row, col = 1, 0
    for name, img in results.items():
        ax[row, col].imshow(to_pil_image(img[0]))
        ax[row, col].set_title(f"Style Weight: {name}")

        col += 1
        if col > 2:
            col = 0
            row += 1

    plt.tight_layout()
    plt.show()


In [54]:
import os

def load_images_from_folder(folder_path, max_size=None):
    images = {}
    for filename in os.listdir(folder_path):
        if filename.lower().endswith((".jpg", ".jpeg", ".png")):
            path = os.path.join(folder_path, filename)
            img = load_image(path, max_size)
            images[filename] = img
    return images

In [55]:

def imshow(tensor, title=None):
    image = tensor.clone().detach()
    image = image.squeeze(0)
    image = image.numpy().transpose(1, 2, 0)
    image = image * [0.229, 0.224, 0.225] + [0.485, 0.456, 0.406]
    image = np.clip(image, 0, 1)
    plt.imshow(image)
    if title:
        plt.title(title)
    plt.axis('off')
    plt.show()


In [56]:
'''
# تحميل الصور من المجلدات
content_images = load_images_from_folder("/content/con img", max_size=512)
style_images = load_images_from_folder("/content/style img", max_size=512)

# اختيار الصور
content_img = content_images["natural.png"]
style_img = style_images["dog.jpg"]

# تنفيذ النقل الأسلوبي مع تحسين الأوزان
output_image = style_transfer(
    content_img, style_img,
    num_steps=500,
    style_weight=1e5,  # توازن أفضل بين الأسلوب والمحتوى
    content_weight=1
)
imshow(output_image, title="Stylized Result - Balanced Weights")
'''

'\n# تحميل الصور من المجلدات\ncontent_images = load_images_from_folder("/content/con img", max_size=512)\nstyle_images = load_images_from_folder("/content/style img", max_size=512)\n\n# اختيار الصور\ncontent_img = content_images["natural.png"]\nstyle_img = style_images["dog.jpg"]\n\n# تنفيذ النقل الأسلوبي مع تحسين الأوزان\noutput_image = style_transfer(\n    content_img, style_img,\n    num_steps=500,\n    style_weight=1e5,  # توازن أفضل بين الأسلوب والمحتوى\n    content_weight=1\n)\nimshow(output_image, title="Stylized Result - Balanced Weights")\n'

In [57]:
'''
# تحميل الصور من المجلدات
content_images = load_images_from_folder("/content/con img", max_size=512)
style_images = load_images_from_folder("/content/style img", max_size=512)

# اختيار الصور
content_img = content_images["portrate.png"]
style_img = style_images["Portrait – Picasso.png"]

# تنفيذ النقل الأسلوبي مع تحسين الأوزان
output_image = style_transfer(
    content_img, style_img,
    num_steps=500,
    style_weight=1e5,  # توازن أفضل بين الأسلوب والمحتوى
    content_weight=1
)
imshow(output_image, title="Stylized Result - Balanced Weights")"""
'''

'\n# تحميل الصور من المجلدات\ncontent_images = load_images_from_folder("/content/con img", max_size=512)\nstyle_images = load_images_from_folder("/content/style img", max_size=512)\n\n# اختيار الصور\ncontent_img = content_images["portrate.png"]\nstyle_img = style_images["Portrait – Picasso.png"]\n\n# تنفيذ النقل الأسلوبي مع تحسين الأوزان\noutput_image = style_transfer(\n    content_img, style_img,\n    num_steps=500,\n    style_weight=1e5,  # توازن أفضل بين الأسلوب والمحتوى\n    content_weight=1\n)\nimshow(output_image, title="Stylized Result - Balanced Weights")"""\n'

In [ ]:
selected_pairs = [
    ("beach.png", "Water Lilies – Monet.jpg"),
    ("city.jpg", "Starry Night – Van Gogh.jpg"),
    ("natural.png", "Mosaic Style.jpg"),
    ("portrate.png", "Portrait – Picasso.jpg"),
    ("street.png", "The Scream – Munch.jpg"),
]

import os
output_dir = "/content/stylized_outputs"
os.makedirs(output_dir, exist_ok=True)

content_images = load_images_from_folder("/content/con img", max_size=512)
style_images = load_images_from_folder("/content/style img", max_size=512)


for content_name, style_name in selected_pairs:
    print(f"Processing: {content_name} + {style_name}")
    # Access content_images and style_images
    content_img = content_images[content_name]
    style_img = style_images[style_name]
    output = style_transfer(
        content_img, style_img,
        num_steps=300, style_weight=1e5, content_weight=1
    )

    result = output.clone().detach().squeeze(0)
    result = result.cpu().numpy().transpose(1, 2, 0)
    result = result * [0.229, 0.224, 0.225] + [0.485, 0.456, 0.406]
    result = np.clip(result, 0, 1)
    from PIL import Image
    result_img = Image.fromarray((result * 255).astype('uint8'))
    save_path = os.path.join(output_dir, f"{content_name.split('.')[0]}_with_{style_name.split('.')[0]}.jpg")
    result_img.save(save_path)
    print(f"Saved to {save_path}\n")

Processing: beach.png + Water Lilies – Monet.jpg


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth
100%|██████████| 548M/548M [00:02<00:00, 195MB/s]


Step 50: Content Loss = 0.4043, Style Loss = 0.0000
Step 100: Content Loss = 0.3512, Style Loss = 0.0000
Step 150: Content Loss = 0.3266, Style Loss = 0.0000
Step 200: Content Loss = 0.3116, Style Loss = 0.0000
Step 250: Content Loss = 0.3004, Style Loss = 0.0000
Step 300: Content Loss = 0.2930, Style Loss = 0.0000
Saved to /content/stylized_outputs/beach_with_Water Lilies – Monet.jpg

Processing: city.jpg + Starry Night – Van Gogh.jpg
Step 50: Content Loss = 5.1864, Style Loss = 0.0001
Step 100: Content Loss = 4.3684, Style Loss = 0.0000
